In [1]:
import selenium
from selenium import webdriver
import time
import pandas as pd

In [2]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

In [3]:
from selenium.webdriver.common.keys import Keys

## 1. Write a python program which searches all the product under a particular product from www.amazon.in. The product name to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

## 2. In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a dataframe and csv. In case if any product vertical has less than 3 pages in search results then scrape all the products available under that product vertical. Details to be scraped are: "Brand Name", "Name of the Product", "Rating", "No. of Ratings", "Price", "Return/Exchange", "Expected Delivery", "Availability", "Other Details" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [3]:
# Function Definition
def enter_product(url):
    driver=webdriver.Chrome("C:\\chromedriver.exe")
    driver.get(url)
    time.sleep(8)
    
    product_name=input("Enter Product: ")
    
    search_bar=driver.find_element_by_id('twotabsearchtextbox')
    search_bar.send_keys(product_name)
    search_butn=driver.find_element_by_id('nav-search-submit-button')
    search_butn.click()
    
#     extracting products url from page-1 to page-3
    href=[]
    for page in range(0,3):
        for i in driver.find_elements_by_xpath("//a[@class='a-link-normal a-text-normal']"):
            href.append(i.get_attribute('href'))
        try: # to enter next page   
            nxt_page=driver.find_element_by_xpath("//li[@class='a-last']/a").get_attribute('href')
            driver.get(nxt_page)
            time.sleep(8)
        except NoSuchElementException:
            break
    
#     Let's create empty lists to store necessary information
    brand=[]
    name_of_the_product=[]
    price=[]
    rating=[]
    no_of_ratings=[]
    return_exg=[]
    expected_delivery=[]
    availability=[]
    other_details=[]
    
    for url in href:
        driver.get(url)
        delay=10
        WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.CLASS_NAME,'a-span12')))

    #     extracting brand
        try:
            td=[]
            for i in driver.find_elements_by_xpath("//td[@class='a-size-base prodDetAttrValue']"):
                td.append(i)
            brand.append(td[0].text)
        except NoSuchElementException:
            brand.append('_')

    #     extracting name_of_the_product
        try:
            n=driver.find_element_by_xpath("//span[@id='productTitle']")
            name_of_the_product.append(n.text)
        except NoSuchElementException:
            name_of_the_product.append('-')
        
#         extracting price
        try:
            p=driver.find_element_by_xpath("//td[@class='a-span12']/span[1]")
            price.append(p.text)
        except NoSuchElementException:
            price.append('-')

    #     extracting rating
        try:
            r=driver.find_element_by_xpath("//span[@id='acrPopover']")
            rating.append(r.get_attribute('title'))
        except NoSuchElementException:
            rating.append('_')

    #     extracting no_of_ratings
        try:
            nr=driver.find_element_by_xpath("//span[@id='acrCustomerReviewText']")
            no_of_ratings.append(nr.text)
        except NoSuchElementException:
            no_of_ratings.append('_')

    #     extracting return/exchange
        try:
            re=[]
            for i in driver.find_elements_by_xpath("//div[@class='a-section a-spacing-none icon-content']/a"):
                re.append(i)
            return_exg.append(re[0].text)
        except NoSuchElementException:
            return_exg.append('_')

    #     extracting expected_delivery
        try:
            ed=driver.find_element_by_xpath("//div[@id='ddmDeliveryMessage']/b")
            expected_delivery.append(ed.text)
        except NoSuchElementException:
            expected_delivery.append('-')

    #     extracting availability
        try:
            a=driver.find_element_by_xpath("//div[@id='availability']/span")
            availability.append(a.text)
        except NoSuchElementException:
            availability.append('-')

    #     extracting other_details
        try:
            od=driver.find_element_by_xpath("//table[@id='productDetails_techSpec_section_1']/tbody")
            other_details.append(od.text.replace('\n',''))
        except NoSuchElementException:
            other_details.append('-')
    
    df=pd.DataFrame({})
    df['Brand']=brand
    df['Name_of_the_Product']=name_of_the_product
    df['Rating']=rating
    df['No_of_Ratings']=no_of_ratings
    df['Price']=price
    df['Return_Exg']=return_exg
    df['Expected_Delivery']=expected_delivery
    df['Availability']=availability
    df['Other_Details']=other_details
    df['Product URL']=href
    
    df.to_csv('product_info.csv',index=False)
    
    driver.close()
    
    return df


# Calling Function
enter_product('https://www.amazon.in/')

Enter Product: tv


,Brand,Name_of_the_Product,Rating,No_of_Ratings,Price,Return_Exg,Expected_Delivery,Availability,Other_Details,Product URL
0,AmazonBasics,AmazonBasics 127cm (50 inches) Fire TV Edition...,4.2 out of 5 stars,653 ratings,"₹ 34,999.00",10 Days Replacement,"Monday, Apr 26, 8AM-5PM",In stock.,Brand AmazonBasicsManufacturer Radiant Applian...,https://www.amazon.in/gp/slredirect/picassoRed...
1,OnePlus,OnePlus 80 cm (32 inches) Y Series HD Ready LE...,4.0 out of 5 stars,"11,790 ratings","₹ 14,999.00",10 Days Replacement,"Monday, Apr 26, 8AM-5PM",In stock.,Brand OnePlusManufacturer Radiant Appliances a...,https://www.amazon.in/gp/slredirect/picassoRed...
2,MI,Mi 4A PRO 80 cm (32 inches) HD Ready Android L...,4.2 out of 5 stars,"37,275 ratings","₹ 14,499.00",10 Days Replacement,"Monday, Apr 26, 8AM-5PM",In stock.,"Brand MIManufacturer Xiaomi, 18001036286Model ...",https://www.amazon.in/Mi-inches-Ready-Android-...
3,EAirtec,eAirtec 61 cms (24 inches) HD Ready LED TV 24D...,4.0 out of 5 stars,"3,651 ratings","₹ 7,499.00",10 Days Replacement,"Monday, Apr 26, 8AM-5PM",In stock.,Brand EAirtecManufacturer Airtec Electrovision...,https://www.amazon.in/eAirtec-inches-Ready-24D...
4,OnePlus,OnePlus 80 cm (32 inches) Y Series HD Ready LE...,4.0 out of 5 stars,"11,790 ratings","₹ 14,999.00",10 Days Replacement,"Monday, Apr 26, 8AM-5PM",In stock.,Brand OnePlusManufacturer Radiant Appliances a...,https://www.amazon.in/OnePlus-inches-Ready-And...
...,...,...,...,...,...,...,...,...,...,...
64,EAirtec,eAirtec 81 cm (32 Inches) HD Ready Smart LED T...,4.1 out of 5 stars,354 ratings,"₹ 11,999.00",10 Days Replacement,"Monday, Apr 26, 8AM-5PM",In stock.,Brand EAirtecManufacturer AIRTEC ELECTROVISION...,https://www.amazon.in/eAirtec-32-Android-SM-VC...
65,LG,LG 108 cm (43 inches) 4K Ultra HD Smart LED TV...,4.4 out of 5 stars,"3,399 ratings","₹ 34,999.00",10 Days Replacement,"Monday, Apr 26, 8AM-5PM",In stock.,Brand LGManufacturer LG Electronics India Pvt....,https://www.amazon.in/LG-inches-Smart-43UM7290...
66,TCL,TCL 165.1 cm (65 inches) 4K Ultra HD Certified...,_,_,"₹ 59,999.00",10 Days Replacement,"Monday, Apr 26, 8AM-5PM",In stock.,"Brand TCLManufacturer Bhagwati Products Ltd., ...",https://www.amazon.in/TCL-inches-Certified-And...
67,Sanyo,Sanyo 126 cm (50 inches) Kaizen Series 4K Ultr...,4.4 out of 5 stars,"1,495 ratings","₹ 34,999.00",10 Days Replacement,"Monday, Apr 26, 8AM-5PM",In stock.,"Brand SanyoManufacturer Panasonic, Panasonic I...",https://www.amazon.in/gp/slredirect/picassoRed...


## 3. Write a python program to access the search bar and search button on images.google.com and scrape 100 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’.

In [4]:
driver=webdriver.Chrome("C:\\chromedriver.exe")
driver.get('https://images.google.com/?gws_rd=ssl')
time.sleep(8)
search_bar=driver.find_element_by_xpath("//input[@class='gLFyf gsfi']")
search_bar.send_keys('fruits')
search_butn=driver.find_element_by_xpath("//button[@class='Tg7LZd']")
search_butn.click()

In [5]:
fruits_img=[]
for i in driver.find_elements_by_xpath("//img[@class='rg_i Q4LuWd']"):
    fruits_img.append(i.get_attribute('src'))

In [6]:
driver.get('https://images.google.com/?gws_rd=ssl')
time.sleep(8)
search_bar=driver.find_element_by_xpath("//input[@class='gLFyf gsfi']")
search_bar.send_keys('cars')
search_butn=driver.find_element_by_xpath("//button[@class='Tg7LZd']")
search_butn.click()

In [7]:
cars_img=[]
for i in driver.find_elements_by_xpath("//img[@class='rg_i Q4LuWd']"):
    cars_img.append(i.get_attribute('src'))

In [8]:
driver.get('https://images.google.com/?gws_rd=ssl')
time.sleep(8)
search_bar=driver.find_element_by_xpath("//input[@class='gLFyf gsfi']")
search_bar.send_keys('Machine Learning')
search_butn=driver.find_element_by_xpath("//button[@class='Tg7LZd']")
search_butn.click()

In [9]:
ml_img=[]
for i in driver.find_elements_by_xpath("//img[@class='rg_i Q4LuWd']"):
    ml_img.append(i.get_attribute('src'))

In [10]:
df=pd.DataFrame({})
df['Fruits']=fruits_img[:100]
df['Cars']=cars_img[:100]
df['Machine Learning']=ml_img[:100]

In [11]:
driver.close()

In [12]:
df.to_csv('img.csv',index=False)

In [13]:
df

,Fruits,Cars,Machine Learning
0,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
1,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
2,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA..."
3,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA..."
4,"data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQA...","data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAA..."
...,...,...,...
95,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...
96,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...
97,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...
98,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...,https://encrypted-tbn0.gstatic.com/images?q=tb...


## 4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Display Resolution”, “Processor”, “Processor Cores”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [3]:
# Function Definition
def smartphone(url):
    driver=webdriver.Chrome("C:\\chromedriver.exe")
    driver.get(url)
    time.sleep(8)
    
    close=driver.find_element_by_xpath("//button[@class='_2KpZ6l _2doB4z']")
    close.click()
    
    enter_mobile=driver.find_element_by_xpath("//input[@title='Search for products, brands and more']")
    enter_mobile.send_keys('google pixel 4a')
    search_butn=driver.find_element_by_xpath("//button[@class='L0Z3Pu']")
    search_butn.click()
    time.sleep(8)
    
#     extracting smartphones href from page-1
    href=[]
    for i in driver.find_elements_by_xpath("//a[@class='_1fQZEK']"):
        href.append(i.get_attribute('href'))
    
#     Let's create empty lists to store necessary information
    smartphone_name=[]
    colour=[]
    ram=[]
    rom=[]
    primary_camera=[]
    secondary_camera=[]
    display_size=[]
    display_resolution=[]
    processor=[]
    processor_cores=[]
    battery_capacity=[]
    price=[]
    brand=[]
    
    for url in href:
        driver.get(url)
        delay=10
        WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.CLASS_NAME,'B_NuCI')))
        
        read_more=driver.find_element_by_xpath("//button[@class='_2KpZ6l _1FH0tX']")
        read_more.click()
        time.sleep(8)
        
#         extracting smartphone_name
        try:
            n=driver.find_element_by_xpath("//h1[@class='yhB1nd']/span")
            smartphone_name.append(n.text)
        except NoSuchElementException:
            smartphone_name.append('-')
        
#         extracting colour
        try:
            elements=[]
            for i in driver.find_elements_by_xpath("//li[@class='_21lJbe']"):
                elements.append(i)
            colour.append(elements[3].text)
        except NoSuchElementException:
            colour.append('-')
        
#         extracting ram
        try:
            elements=[]
            for i in driver.find_elements_by_xpath("//div[@class='_3k-BhJ']/table/tbody/tr[2]/td[2]/ul/li"):
                elements.append(i)
            ram.append(elements[3].text)
        except NoSuchElementException:
            ram.append('-')
        
#         extracting rom
        try:
            elements=[]
            for i in driver.find_elements_by_xpath("//div[@class='_3k-BhJ']/table/tbody/tr[1]/td[2]/ul/li"):
                elements.append(i)
            rom.append(elements[3].text)
        except NoSuchElementException:
            rom.append('-')
        
#         extracting primary_camera
        try:
            elements=[]
            for i in driver.find_elements_by_xpath("//table[@class='_14cfVK']/tbody/tr[2]/td[2]/ul/li"):
                elements.append(i)
            primary_camera.append(elements[4].text)
        except NoSuchElementException:
            primary_camera.append('-')
        
#         extracting secondary_camera
        try:
            elements=[]
            for i in driver.find_elements_by_xpath("//table[@class='_14cfVK']/tbody/tr[5]/td[2]/ul/li"):
                elements.append(i)
            secondary_camera.append(elements[3].text)
        except NoSuchElementException:
            secondary_camera.append('-')
        
#         extracting display_size
        try:
            elements=[]
            for i in driver.find_elements_by_xpath("//table[@class='_14cfVK']/tbody/tr[1]/td[2]/ul/li"):
                elements.append(i)
            display_size.append(elements[1].text)
        except NoSuchElementException:
            display_size.append('-')
        
#         extracting display_resolution
        try:
            elements=[]
            for i in driver.find_elements_by_xpath("//table[@class='_14cfVK']/tbody/tr[2]/td[2]/ul/li"):
                elements.append(i)
            display_resolution.append(elements[1].text)
        except NoSuchElementException:
            display_resolution.append('-')
        
#         extracting processor
        try:
            elements=[]
            for i in driver.find_elements_by_xpath("//table[@class='_14cfVK']/tbody/tr[2]/td[2]/ul/li"):
                elements.append(i)
            processor.append(elements[2].text)
        except NoSuchElementException:
            processor.append('-')
        
#         extracting processor_cores
        try:
            elements=[]
            for i in driver.find_elements_by_xpath("//table[@class='_14cfVK']/tbody/tr[3]/td[2]/ul/li"):
                elements.append(i)
            processor_cores.append(elements[2].text)
        except NoSuchElementException:
            processor_cores.append('-')
        
#         extracting battery_capacity
        try:
            elements=[]
            for i in driver.find_elements_by_xpath("//table[@class='_14cfVK']/tbody/tr[1]/td[2]/ul/li"):
                elements.append(i)
            battery_capacity.append(elements[7].text)
        except NoSuchElementException:
            battery_capacity.append('-')
        
#         extracting price
        try:
            p=driver.find_element_by_xpath("//div[@class='_25b18c']/div")
            price.append(p.text)
        except NoSuchElementException:
            price.append('-')
        
#         extracting brand
        try:
            info=driver.find_element_by_xpath("//div[@class='_1JDTUN']")
            info.click()
            time.sleep(10)
            b=driver.find_element_by_xpath("//li[@class='_3_27HS']")
            brand.append(b.text)
        except NoSuchElementException:
            brand.append('-')
    
    df=pd.DataFrame({})
    df['Brand']=brand
    df['Colour']=colour
    df['Ram']=ram
    df['Rom']=rom
    df['Primary_camera']=primary_camera
    df['Secondary_camera']=secondary_camera
    df['Display_size']=display_size
    df['Display_resolution']=display_resolution
    df['processor']=processor
    df['processor_cores']=processor_cores
    df['battery_capacity']=battery_capacity
    df['price']=price
    df['Product_url']=href
    
    df.to_csv('smartphone.csv',index=False)
    
    driver.close()
    
    return df

# Calling Function
smartphone('https://www.flipkart.com/')

,Brand,Colour,Ram,Rom,Primary_camera,Secondary_camera,Display_size,Display_resolution,processor,processor_cores,battery_capacity,price,Product_url
0,"Google LLC, 1600 Amphitheatre Pkwy, Mountain v...",Just Black,6 GB,128 GB,12.2MP Rear Camera,8MP Front Camera,14.76 cm (5.81 inch),2340 x 1080 Pixels,Qualcomm Snapdragon 730G,Octa Core,Yes,"₹31,999",https://www.flipkart.com/google-pixel-4a-just-...
1,"Google LLC, 1600 Amphitheatre Pkwy, Mountain v...",Clearly White,4 GB,64 GB,12.2MP Rear Camera,8MP Front Camera,14.22 cm (5.6 inch),2220 x 1080 pixels,Qualcomm Snapdragon 670,Octa Core,3000 mAh,"₹39,999",https://www.flipkart.com/google-pixel-3a-clear...
2,"Google LLC, 1600 Amphitheatre Pkwy, Mountain v...",Just Black,4 GB,64 GB,12.2MP Rear Camera,8MP Front Camera,14.22 cm (5.6 inch),2220 x 1080 pixels,Qualcomm Snapdragon 670,Octa Core,3000 mAh,"₹39,999",https://www.flipkart.com/google-pixel-3a-just-...
3,"Google LLC, 1600 Amphitheatre Pkwy, Mountain v...",Clearly White,4 GB,64 GB,12.2MP Rear Camera,8MP + 8MP Dual Front Camera,16.0 cm (6.3 inch),2960 x 1440 pixels,Qualcomm Snapdragon 845 64-bit,Octa Core,3430 mAh,"₹83,000",https://www.flipkart.com/google-pixel-3-xl-cle...
4,"Google LLC, 1600 Amphitheatre Pkwy, Mountain v...",Just Black,4 GB,64 GB,12.2MP Rear Camera,8MP Front Camera,15.24 cm (6 inch),2160 x 1080 pixels,Qualcomm Snapdragon 670,Octa Core,3700 mAh,"₹44,999",https://www.flipkart.com/google-pixel-3a-xl-ju...
5,"Google LLC, 1600 Amphitheatre Pkwy, Mountain v...",Just Black,4 GB,64 GB,12.2MP Rear Camera,8MP Front Camera,12.7 cm (5 inch),1920 x 1080 Pixels,Qualcomm Snapdragon 835 64-bit Octa Core,Octa Core,2700 mAh,"₹61,000",https://www.flipkart.com/google-pixel-2-just-b...
6,"Google LLC, 1600 Amphitheatre Pkwy, Mountain v...",Kinda Blue,4 GB,64 GB,12.2MP Rear Camera,8MP Front Camera,12.7 cm (5 inch),1920 x 1080 Pixels,Qualcomm Snapdragon 835 64-bit Octa Core,Octa Core,2700 mAh,"₹61,000",https://www.flipkart.com/google-pixel-2-kinda-...
7,"Google LLC, 1600 Amphitheatre Pkwy, Mountain v...",Black & White,4 GB,64 GB,12.2MP Rear Camera,8MP Front Camera,15.24 cm (6 inch),2880 x 1440 Pixels,Qualcomm Snapdragon 835 64-bit Octa Core,Octa Core,3520 mAh,"₹45,499",https://www.flipkart.com/google-pixel-2-xl-bla...
8,"Google LLC, 1600 Amphitheatre Pkwy, Mountain v...",Smartphones,4 GB,128 GB,12.2MP Rear Camera,8MP Front Camera,15.24 cm (6 inch),2880 x 1440 Pixels,Qualcomm Snapdragon 835 64-bit Octa Core,Octa Core,3520 mAh,"₹82,000",https://www.flipkart.com/google-pixel-2-xl-jus...
9,"Google LLC, 1600 Amphitheatre Pkwy, Mountain v...",Clearly White,4 GB,128 GB,12.2MP Rear Camera,8MP Front Camera,12.7 cm (5 inch),1920 x 1080 Pixels,Qualcomm Snapdragon 835 64-bit Octa Core,Octa Core,2700 mAh,"₹70,000",https://www.flipkart.com/google-pixel-2-clearl...


## 5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [18]:
driver=webdriver.Chrome('C:\\chromedriver.exe')
driver.get('https://www.google.co.in/maps/@10.54063,76.1283185,7z')
time.sleep(8)

search_bar=driver.find_element_by_xpath("//div[@id='gs_lc50']/input")
search_bar.send_keys('Bengaluru')
search_butn=driver.find_element_by_xpath("//div[@class='searchbox-searchbutton-container']/button")
search_butn.click()
time.sleep(8)

zoom_in=driver.find_element_by_xpath("//div[@jsaction='blur:zoom.onZoomInOrTooltipBlur']/button")
zoom_in.click()
time.sleep(4)

In [19]:
# extracting latitude,longitude
lat_lon=driver.find_element_by_xpath("//button[@vet='10547']").get_attribute('aria-label')
print(lat_lon)

# extracting area
area=driver.find_element_by_xpath("//div[@class='widget-reveal-card-address-line g1qtdkppGXY__widget-reveal-card-bold']")
print(area.text)

# extracting city
city=driver.find_element_by_xpath("//div[@class='widget-reveal-card-address-line']")
print(city.text)

12.967231, 77.748983
Whitefield
Bengaluru, Karnataka


In [20]:
info={'Latitude,Longitude':lat_lon,'Area':area.text,'City':city.text}

df=pd.DataFrame(info,index=[0])

df

,"Latitude,Longitude",Area,City
0,"12.967231, 77.748983",Whitefield,"Bengaluru, Karnataka"


In [21]:
driver.close()

## 6. Write a program to scrap details of all the funding deals for second quarter (i.e. July 20 – September 20) from trak.in.

In [7]:
driver=webdriver.Chrome("C:\\chromedriver.exe")
driver.get('https://trak.in/')
time.sleep(8)

funding_deals=driver.find_element_by_xpath("//ul[@class='sub-menu bs-pretty-tabs-elements']/li/a").get_attribute('href')
driver.get(funding_deals)
time.sleep(8)

In [8]:
# july month
j=[]
for i in driver.find_elements_by_xpath("//div[@id='tablepress-48_wrapper']/div[3]/div[2]/table/tbody/tr/td"):
    j.append(i.text)

july=pd.DataFrame({})
july['Date']=j[1::9]
july['Startup Name']=j[2::9]
july['Industry/Vertical']=j[3::9]
july['Sub-vertical']=j[4::9]
july['City']=j[5::9]
july['Investors Name']=j[6::9]
july['Investment Type']=j[7::9]
july['Amount(USD)']=j[8::9]

july

,Date,Startup Name,Industry/Vertical,Sub-vertical,City,Investors Name,Investment Type,Amount(USD)
0,15/07/2020,Flipkart,E-commerce,E-commerce,Bangalore,Walmart Inc,M&A,"1,200,000,000"
1,16/07/2020,Vedantu,EduTech,Online Tutoring,Bangalore,Coatue Management,Series D,"100,000,000"
2,16/07/2020,Crio,EduTech,Learning Platform for Developers,Bangalore,021 Capital,pre-Series A,"934,160"
3,14/07/2020,goDutch,FinTech,Group Payments,Mumbai,"Matrix India,Y Combinator, Global Founders Cap...",Seed,"1,700,000"
4,13/07/2020,Mystifly,Airfare Marketplace,"Ticketing, Airline Retailing, and Post-Ticketi...",Singapore and Bangalore,Recruit Co. Ltd.,pre-Series B,"3,300,000"
5,09/07/2020,JetSynthesys,Gaming and Entertainment,Gaming and Entertainment,Pune,Adar Poonawalla and Kris Gopalakrishnan.,Venture-Series Unknown,"400,000"
6,10/07/2020,gigIndia,Marketplace,"Crowd Sourcing, Freelance",Pune,Incubate Fund India and Beyond Next Ventures,pre-Series A,"974,200"
7,15/07/2020,PumPumPum,Automotive Rental,Used Car-leasing platform,Gurgaon,Early Adapters Syndicate,Seed,"292,800"
8,14/07/2020,FLYX,OTT Player,Streaming Social Network,New York and Delhi,"Raj Mishra, founder of AIT Global Inc",pre-Seed,"200,000"
9,13/07/2020,Open Appliances Pvt. Ltd.,Information Technology,Internet-of-Things Security Solutions,Bangalore,Unicorn India Ventures,Venture-Series Unknown,"500,000"


In [9]:
# august month
a=[]
for i in driver.find_elements_by_xpath("//div[@id='tablepress-49_wrapper']/div[3]/div[2]/table/tbody/tr/td"):
    a.append(i.text)
nxt=driver.find_element_by_xpath("//a[@id='tablepress-49_next']")
nxt.click()
for i in driver.find_elements_by_xpath("//div[@id='tablepress-49_wrapper']/div[3]/div[2]/table/tbody/tr/td"):
    a.append(i.text)

aug=pd.DataFrame({})
aug['Date']=a[1::9]
aug['Startup Name']=a[2::9]
aug['Industry/Vertical']=a[3::9]
aug['Sub-vertical']=a[4::9]
aug['City']=a[5::9]
aug['Investors Name']=a[6::9]
aug['Investment Type']=a[7::9]
aug['Amount(USD)']=a[8::9]

aug

,Date,Startup Name,Industry/Vertical,Sub-vertical,City,Investors Name,Investment Type,Amount(USD)
0,15/08/2020,Practo,HealthTech,Health care and Wellness,Bangalore,A1A Company,Series F,"32,000,000"
1,13/08/2020,Medlife,E-commerce,Online Pharmacy,Bangalore,Prasid Uno Family Trust and SC Credit Fund,,"23,000,000"
2,13/08/2020,HungerBox,FoodTech,Online Food Delivery Service,Bangalore,"One97, Sabre Partners Trust, Pratithi Investme...",Series D1,"1,560,000"
3,04/08/2020,Dunzo,Hyper-local Logistics,Online Delivery Services,Bangalore,Existing Backers,In Progress,"30,000,000"
4,11/08/2020,Terra.do,EduTech,"Online Climate School, E-learning","Stanford, California,","Stanford Angels and Entrepreneurs (India), BEE...",Seed,"1,400,000"
5,12/08/2020,Classplus,EduTech,"E-learning, Online Tutoring",Noida,Falcon Edge,In Progress,"upto 15,000,000"
6,14/08/2020,Niyo,FinTech,Financial Services,Bangalore,Niyo Solutions Inc.,,"6,000,000"
7,10/08/2020,ZestMoney,FinTech,Financial Services,Bangalore,Primrose Hills Ventures,,"10,670,000"
8,07/08/2020,FreshToHome,E-commerce,Food Delivery,Bangalore,Ascent Capital,Venture,"16,200,000"
9,13/08/2020,Eduvanz,FinTech,Financial Services,Mumbai,"Sequoia India, Unitus",Series A,"5,000,000"


In [10]:
# september month
s=[]
for i in driver.find_elements_by_xpath("//div[@id='tablepress-50_wrapper']/div[3]/div[2]/table/tbody/tr/td"):
    s.append(i.text)
nxt=driver.find_element_by_xpath("//a[@id='tablepress-50_next']")
nxt.click()
for i in driver.find_elements_by_xpath("//div[@id='tablepress-50_wrapper']/div[3]/div[2]/table/tbody/tr/td"):
    s.append(i.text)
    
sep=pd.DataFrame({})
sep['Date']=s[1::9]
sep['Startup Name']=s[2::9]
sep['Industry/Vertical']=s[3::9]
sep['Sub-vertical']=s[4::9]
sep['City']=s[5::9]
sep['Investors Name']=s[6::9]
sep['Investment Type']=s[7::9]
sep['Amount(USD)']=s[8::9]

sep

,Date,Startup Name,Industry/Vertical,Sub-vertical,City,Investors Name,Investment Type,Amount(USD)
0,08/09/2020,Byju’s,EduTech,Online Tutoring,Bangalore,"Silver Lake, Tiger Global, General Atlantic an...",Private Equity,"500,000,000"
1,12/09/2020,mCaffeine,Personal Care,Skincare & Haircare,Mumbai,"Amicus Capital Private Equity I LLP, Amicus Ca...",Series B,"3,000,000"
2,09/09/2020,Qshala,EduTech,Online Curiosity Platform for Kids,Bangalore,Rainmatter Capital,Angel,"370,000"
3,02/09/2020,Winzo,Online Gaming,Online Gaming,New Delhi,"Kalaari Capital Partners, IndigoEdge Managemen...",Series B,"15,500,000"
4,09/09/2020,Hippo Video,Video Customer Experience(CX) Platform,Video Customer Experience(CX) Platform,"Newark, Delaware, United States of Amercia","Alpha Wave Incubation, Exfinity Venture Partne...",Series A,"4,500,000"
5,07/09/2020,Melorra,E-commerce,Online Jewelry Store,Bangalore,"Shadow Holdings, Lightbox.",Debt Financing,"upto 8,900,000"
6,07/09/2020,1mg,E-commerce,Online Pharmacy,Gurgaon,"Gaja Capital, Tata Capital, Partners Group",In Progress,"100,000,000"
7,31/08/2020,mfine,HealthTech,On-Demand Healthcare Services,Bangalore,Caretech Pte Inc,Series B,"5,400,000"
8,31/08/2020,Apna,Human Resources,Recruitment Platform,Bangalore,Lightspeed India and Sequoia Capital India,Series A,"8,000,000"
9,03/09/2020,Railofy,Transportation,WL & RAC protection platform,Mumbai,Chiratae Ventures,Seed,"950,000"


In [11]:
driver.close()

In [12]:
df=pd.concat([july,aug,sep],ignore_index=True)

In [13]:
df.to_csv('funding_deals.csv',index=False)

In [14]:
df

,Date,Startup Name,Industry/Vertical,Sub-vertical,City,Investors Name,Investment Type,Amount(USD)
0,15/07/2020,Flipkart,E-commerce,E-commerce,Bangalore,Walmart Inc,M&A,"1,200,000,000"
1,16/07/2020,Vedantu,EduTech,Online Tutoring,Bangalore,Coatue Management,Series D,"100,000,000"
2,16/07/2020,Crio,EduTech,Learning Platform for Developers,Bangalore,021 Capital,pre-Series A,"934,160"
3,14/07/2020,goDutch,FinTech,Group Payments,Mumbai,"Matrix India,Y Combinator, Global Founders Cap...",Seed,"1,700,000"
4,13/07/2020,Mystifly,Airfare Marketplace,"Ticketing, Airline Retailing, and Post-Ticketi...",Singapore and Bangalore,Recruit Co. Ltd.,pre-Series B,"3,300,000"
5,09/07/2020,JetSynthesys,Gaming and Entertainment,Gaming and Entertainment,Pune,Adar Poonawalla and Kris Gopalakrishnan.,Venture-Series Unknown,"400,000"
6,10/07/2020,gigIndia,Marketplace,"Crowd Sourcing, Freelance",Pune,Incubate Fund India and Beyond Next Ventures,pre-Series A,"974,200"
7,15/07/2020,PumPumPum,Automotive Rental,Used Car-leasing platform,Gurgaon,Early Adapters Syndicate,Seed,"292,800"
8,14/07/2020,FLYX,OTT Player,Streaming Social Network,New York and Delhi,"Raj Mishra, founder of AIT Global Inc",pre-Seed,"200,000"
9,13/07/2020,Open Appliances Pvt. Ltd.,Information Technology,Internet-of-Things Security Solutions,Bangalore,Unicorn India Ventures,Venture-Series Unknown,"500,000"


## 7. Write a program to scrap all the available details of top 10 gaming laptops from digit.in.

In [4]:
# Function Definition
def laptops(url):
    driver=webdriver.Chrome("C:\\chromedriver.exe")
    driver.get(url)
    time.sleep(8)
    
    search_butn=driver.find_element_by_xpath("//div[@class='search']/a")
    search_butn.click()
    time.sleep(4)
    search_bar=driver.find_element_by_xpath("//div[@class='modal-body']/input")
    search_bar.send_keys('gaming laptops')
    search_bar=driver.find_element_by_xpath("//div[@class='modal-body']/input")
    search_bar.send_keys(Keys.ENTER)
    time.sleep(8)
    
    top_10=driver.find_element_by_xpath("//form[@id='contenttypeId']/ul/li[5]/label/input")
    top_10.click()
    delay=10
    WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.CLASS_NAME,'searchPage')))
    
    href=[]
    for i in driver.find_elements_by_xpath("//div[@class='searchPage']/a"):
        href.append(i.get_attribute('href'))
    driver.get(href[0])
    delay=10
    WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.CLASS_NAME,'Spcs-details')))
    
#     Let's create empty lists to store necessary information
    name=[]
    os=[]
    display	=[]
    processor=[]
    memory=[]
    weight=[]
    dimension=[]
    graphics=[]
    price=[]
    
#     extracting product name
    for i in driver.find_elements_by_xpath("//h3"):
        name.append(i.text)
    
#     extracting os
    for i in driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr[3]/td[3]"):
        os.append(i.text)
    
#     extracting display
    for i in driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr[4]/td[3]"):
        display.append(i.text)
    
#     extracting processor
    for i in driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr[5]/td[3]"):
        processor.append(i.text)
    
#     extracting memory
    for i in driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr[6]/td[3]"):
        memory.append(i.text)
    
#     extracting weight
    for i in driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr[7]/td[3]"):
        weight.append(i.text)
    
#     extracting dimension
    for i in driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr[8]/td[3]"):
        dimension.append(i.text)
    
#     extracting graphics
    for i in driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr[9]/td[3]"):
        graphics.append(i.text)
    
#     extracting price
    full_specs_href=[]
    for i in driver.find_elements_by_xpath("//span[@class='datahreflink spec']"):
        full_specs_href.append(i.get_attribute('data-href'))
    
    for url in full_specs_href:
        driver.get(url)
        delay=10
        WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.CLASS_NAME,'Block-price')))
        try:
            p=driver.find_element_by_xpath("//div[@class='Onward-price']/div/b")
            price.append(p.text)
        except:
            price.append('-')
    
    df=pd.DataFrame({})
    df['Product Name']=name
    df['Os']=os
    df['Display']=display
    df['Processor']=processor
    df['Memory']=memory
    df['Weight']=weight
    df['Dimension']=dimension
    df['Graphics']=graphics
    df['Price']=price
    
    df.to_csv('laptops.csv',index=False)
    
    driver.close()
    
    return df

laptops('https://www.digit.in/laptops/')

,Product Name,Os,Display,Processor,Memory,Weight,Dimension,Graphics,Price
0,MSI GT76 TITAN DT 9SG,Windows 10 Pro,"17.3"" (UHD 3840x2160)",Intel 9th Gen Core i9-9900K | 5000 MHz,1 TB HDD/64 GBGB DDR4,4.2 kg,397 x 330 x 33~42 mm,NVIDIA GeForce RTX 2080,"379,990"
1,ALIENWARE 17 AREA-51M,Windows 10 Pro,"17.3"" (FHD (1920 x 1080))",Intel 9th Gen Core i9-9900K | 5000 MHz,1 TB PCIe SSD/32GB DDR4,3.87 Kg,42 mm x 402.6 mm x 319.14 mm,NVIDIA GeForce RTX 2080,"422,000"
2,HP OMEN 15 2020,Windows 10 Home,"15.6"" (1920 x 1080)",Intel i7-10750H 10th Gen | 1.6GHz,512 GB SSD/16 GBGB DDR4,5.40,14.09 x 9.44 x 0.89,Nvidia GeForce GTX 1650Ti,"117,790"
3,ASUS ZEPHYRUS G14,Windows 10 Home,"14"" (1920 x 1080)",AMD 3rd Generation Ryzen 9 | 3.3 GHz,1 TB SSD/16 GBGB DDR4,1.65,32.5 x 22.1 x 1.8,NVIDIA GeForce RTX 2060,"164,990"
4,LENOVO LEGION Y540,Windows 10 Home,"15.6"" (1920 X 1080)",9th Generation Core Intel I7-9750H | 2.6 Ghz,1 TB SSD/8GB DDR4,2.3,365mm x 260mm x 25.9mm,NVIDIA® GeForce RTX™ 2060,"79,990"
5,ASUS ROG ZEPHYRUS G GA502,Windows 10 Home,"15.6"" (1920 x 1080)",AMD Ryzen 7 Quad Core 3750H | 2.3 GHz,512 GB SSD/16GB DDR4,2.2,360 x 252 x 20.4,NVIDIA Geforce GTX 1660 Ti,"79,990"
6,ASUS ROG ZEPHYRUS S GX531,Windows 10 Home,"15.6"" (1920 X 1080)",9th Gen Intel Core i7-8750H | 2.2 GHz,512GB SSD/16 GBGB DDR4,2.1,360 (W) x 268 (D) x 15.35~16.15 (H) mm,NVIDIA® GeForce RTX™ 2070 (Max-Q),"239,990"
7,MSI GT83VR 7RE TITAN SLI,Windows 10 Home 64 bit,"18.4"" (1920 x 1080)",Intel CM238 Core i7-7820HK+CM238 7th Gen | 3.5GHz,1.5 TB SATA/64GB DDR4,5.5,458 x 339 x 69,Dual GTX1070,"349,990"
8,ASUS ROG ZEPHYRUS DUO 15,Windows 10,"15.6"" (3840 x 1100)",Intel Core i7 10th Gen 10875H | NA,512 GB SSD/4 GBGB DDR4,2.4,268.30 x 360.00 x 20.90,NVIDIA GeForce RTX 2070 Max-Q,"244,990"
9,DELL G3,Windows 10 Home Plus,15.6 MP | NA,8th Gen Intel core i5-8300H | 2.3GHz,1TB HDD/8GB DDR4,2.53,22.7 X 380 X 258,NVidia GeForce GTX 1050,"73,900"
